# [Цели исследования](#stage_0_1)

# [Описание данных](#stage_0_2)

# [1. Изучение датасета](#stage_1)

# [2. Предобработка данных](#stage_2)

# [3. Оценка корректности проведения теста](#stage_3)

# [4. Исследователький анализ данных](#stage_4)

## [Промежуточный вывод](#stage_4_1)

# [5. Статистический анализ данных](#stage_5)
## [Промежуточный вывод](#stage_5_1)

# [6. Выводы и рекомендации](#stage_6)

<a name="stage_0_1"></a>

## Цели исследования 

Провести оценку результатов A/B-теста :   
Оценить корректность проведения теста и проанализироваить его результаты.

<a name="stage_0_2"></a>
## Описание данных

    `/datasets/ab_project_marketing_events.csv` — календарь маркетинговых событий на 2020 год; 
    
Структура файла:
`name` — название маркетингового события;  
`regions` — регионы, в которых будет проводиться рекламная кампания;  
`start_dt` — дата начала кампании;  
`finish_dt` — дата завершения кампании.  

`/datasets/final_ab_new_users.csv` — все пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года;  

Структура файла:  
`user_id` — идентификатор пользователя;  
`first_date` — дата регистрации;  
`region` — регион пользователя;  
`device` — устройство, с которого происходила регистрация.  

`/datasets/final_ab_events.csv` — все события новых пользователей в период с 7 декабря 2020 по 4 января 2021 года;  

Структура файла:  
`user_id` — идентификатор пользователя;  
`event_dt` — дата и время события;  
`event_name` — тип события;  
`details` — дополнительные данные о событии. Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.  

`/datasets/final_ab_participants.csv` — таблица участников тестов.  

Структура файла:  
`user_id` — идентификатор пользователя;  
`ab_test` — название теста;  
`group` — группа пользователя.  

In [1]:
#Импортируем нужные библиотеки

import pandas as pd
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
import plotly.express as px
import seaborn as sns
from plotly import graph_objects as go

from scipy import stats as st
import math as mth
from datetime import datetime, timedelta

<a name="stage_0_1"></a>

# 1. Изучение датасета

In [2]:
# загрузим данные и преобразуем значения в столбцах

path = 'C:\\Users\\docto\\Downloads/\\/'

try:
    pm_events = pd.read_csv(path + 'ab_project_marketing_events.csv')
    new_users = pd.read_csv(path + 'final_ab_new_users.csv')
    events = pd.read_csv(path + 'final_ab_events.csv')
    participants = pd.read_csv(path + 'final_ab_participants.csv')
except FileNotFoundError:
    pm_events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv')
    new_users = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_new_users.csv')
    events = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_events.csv')
    participants = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_participants.csv')

In [3]:
#обзор иходных данных
pm_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [4]:
pm_events.sample(5)

,name,regions,start_dt,finish_dt
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
11,Dragon Boat Festival Giveaway,APAC,2020-06-25,2020-07-01


In [5]:
new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


In [6]:
new_users.sample(5)

,user_id,first_date,region,device
23569,724097271BC66C44,2020-12-15,CIS,PC
13472,E764BDD6D61C4207,2020-12-21,EU,Android
54156,ECB05CFCBC67845D,2020-12-13,EU,Android
47246,3278EC17A202C9DB,2020-12-12,EU,iPhone
51377,5E85AF1D667CC793,2020-12-19,EU,PC


In [7]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


In [8]:
events.sample(5)

,user_id,event_dt,event_name,details
188179,0B986C8987511A58,2020-12-19 00:23:02,product_page,NaN
75422,57EEF3B13ABF1FE2,2020-12-13 00:10:46,product_cart,NaN
24488,1EDC34F1BBFDE5D0,2020-12-16 01:52:07,purchase,4.99
109631,24883AB1D58D5F19,2020-12-23 11:21:26,product_cart,NaN
224505,2A3CC2DB6F886374,2020-12-23 02:04:04,product_page,NaN


In [9]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


In [10]:
participants.sample(5)

,user_id,group,ab_test
8632,26D5667724FF2AC9,B,interface_eu_test
17672,2916F3AC42FDDF50,B,interface_eu_test
12270,A44C71535FA3FD7A,A,interface_eu_test
4974,34C31A8D12A1979A,B,recommender_system_test
9291,6C735930F7D9E26D,A,interface_eu_test


При обзоре данных выяснилось, что нам требуется только скорректировать формат некоторых столбцов даты.

<a name="stage_2"></a>
# 2. Предобработка данных

Скорректируем формат даты: 

In [11]:
pm_events['start_dt'] = pd.to_datetime(pm_events['start_dt'])
pm_events['finish_dt'] = pd.to_datetime(pm_events['start_dt'])
pm_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes


In [12]:
events['event_dt'] = pd.to_datetime(events['event_dt'])
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     440317 non-null  object        
 1   event_dt    440317 non-null  datetime64[ns]
 2   event_name  440317 non-null  object        
 3   details     62740 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 13.4+ MB


In [13]:
new_users['first_date'] = pd.to_datetime(new_users['first_date'])
new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     61733 non-null  object        
 1   first_date  61733 non-null  datetime64[ns]
 2   region      61733 non-null  object        
 3   device      61733 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.9+ MB


Формат даты в столбцах успешно скорректирован

## Проверим датасеты на наличие дубликатов

In [14]:
pm_events.duplicated().sum()

0

In [15]:
new_users.duplicated().sum()

0

In [16]:
events.duplicated().sum()

0

In [17]:
participants.duplicated().sum()

0

Явные дубликаты не обнаружены

## Изучение неявных дубликатов

In [42]:
print(f'Столбец "name": {pm_events.name.unique()}')
print(f'Столбец "region": {new_users.region.unique()}')
print(f'Столбец "device": {new_users.device.unique()}')
print(f'Столбец "event_name": {events.event_name.unique()}')
print(f'Столбец "group": {participants.group.unique()}')
print(f'Столбец "ab_test": {participants.ab_test.unique()}')

Столбец "name": ['Christmas&New Year Promo' "St. Valentine's Day Giveaway"
 "St. Patric's Day Promo" 'Easter Promo' '4th of July Promo'
 'Black Friday Ads Campaign' 'Chinese New Year Promo'
 'Labor day (May 1st) Ads Campaign' "International Women's Day Promo"
 'Victory Day CIS (May 9th) Event' 'CIS New Year Gift Lottery'
 'Dragon Boat Festival Giveaway' "Single's Day Gift Promo"
 'Chinese Moon Festival']
Столбец "region": ['EU' 'N.America' 'APAC' 'CIS']
Столбец "device": ['PC' 'Android' 'iPhone' 'Mac']
Столбец "event_name": ['purchase' 'product_cart' 'product_page' 'login']
Столбец "group": ['A' 'B']
Столбец "ab_test": ['recommender_system_test' 'interface_eu_test']


Неявные дубликаты не обнаружены

## Проверим количество пропусков и при необходимости заполним их

In [18]:
pm_events.isna().sum()

name         0
regions      0
start_dt     0
finish_dt    0
dtype: int64

In [19]:
new_users.isna().sum()

user_id       0
first_date    0
region        0
device        0
dtype: int64

In [20]:
events.isna().sum()

user_id            0
event_dt           0
event_name         0
details       377577
dtype: int64

In [21]:
participants.isna().sum()

user_id    0
group      0
ab_test    0
dtype: int64

В датафрейме `events` выявилены пропуски в столбце `details`  - дополнительные данные о событии.  
Эти пропуски не пригодятся нам для дальнейшего анализа поэтому заполнять их не требуется.

<a name="stage_3"></a>
# 3 Оценка корректности проведения теста

## Проверка периода набора пользователей в тест и его соответствие требованиям технического задания

Техническое задание

- Название теста: `recommender_system_test`;  
- Группы: А (контрольная), B (новая платёжная воронка);  
- Дата запуска: 2020-12-07;  
- Дата остановки набора новых пользователей: 2020-12-21;  
- Дата остановки: 2021-01-04;  
- Ожидаемое количество участников теста: 15% новых пользователей из региона EU;  
- Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;  
- Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 5 процентных пунктов:  
    - конверсии в просмотр карточек товаров — событие `product_page`  
    - просмотры корзины — `product_cart`  
    - покупки — `purchase`.  

## Проверка дат событий

Проверим период регистрации пользователей : данные должны соответствовать периоду 14 дней с 7 декабря 2020 года по 21 декабря 2020 года.

In [47]:
min_date = participants.query('ab_test == "recommender_system_test"').merge(new_users, on='user_id', how='left').first_date.min()
max_date = participants.query('ab_test == "recommender_system_test"').merge(new_users, on='user_id', how='left').first_date.max()
print(f'Пользователи проходили регистрацию с {min_date} по {max_date}.')

Пользователи проходили регистрацию с 2020-12-07 00:00:00 по 2020-12-21 00:00:00.


In [49]:
users = new_users.query('first_date <= "2020-12-21"')

События укладываются в заданный период, однако последне событие произошло гораздо раньше окончания периода тестирования, вполне возможно были утерены данные за январь 2021 года.

Проверим период в который пользователи проводили действия:

In [52]:
print(events.event_dt.min())
print(events.event_dt.max())

2020-12-07 00:00:33
2020-12-30 23:36:33


Пользователи проводили действия на пять дней меньше необходимого.

Проверим ожидаемое количество участников теста (6000 человек) и процент новых пользователей из целевого региона EU (15%).

In [55]:
users[users['region']=='EU'].value_counts()

user_id           first_date  region  device 
0001710F4DDB1D1B  2020-12-14  EU      Android    1
AA0D36EF3DBFCC9F  2020-12-09  EU      Mac        1
AA0DB6AAB6CBC024  2020-12-20  EU      iPhone     1
AA116D67C4F48D50  2020-12-12  EU      Android    1
AA168857CA88A885  2020-12-21  EU      Android    1
                                                ..
5535317BAF884BAE  2020-12-14  EU      Android    1
55359CFD29A83E13  2020-12-11  EU      iPhone     1
5538F90C22B9B478  2020-12-21  EU      Mac        1
553BAE96C6EB6240  2020-12-20  EU      Android    1
FFF91B6C5431F375  2020-12-14  EU      PC         1
Length: 42340, dtype: int64

Мы наблюдаем 42340 пользователей из Европы.

In [62]:
eu_users = (participants
    .query('ab_test == "recommender_system_test"')['user_id']
    .reset_index()
    .merge(users
        .query('region == "EU"')[['user_id', 'region']],
        how='left',
        on='user_id'
    )
)

eu_users = eu_users[['user_id', 'region']].dropna()

In [63]:
print('Количество пользователей из Европы составляет',eu_users.user_id.count())
print('Доля новых пользователей из Европы: {:.0%}'.format(len(eu_users)/users.query('region == "EU"')['user_id'].count()))

Количество пользователей из Европы составляет 6351
Доля новых пользователей из Европы: 15%


## Проверка времени проведения теста и пересечение аудитории

Проверим, что время проведения теста (с 07.12.2020 по 04.01.2021) не совпадает с маркетинговыми и другими активностями.

In [64]:
pm_events.query('regions.str.contains("EU") & start_dt >= "2020-12-07" & finish_dt <= "2021.01.04"')

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2020-12-25


В Европе в период проведения теста проводилась кампания "Christmas&New Year Promo" что может исказить результаты теста.

Проверим пересечение пользователей в группах теста

In [65]:
participants.groupby('ab_test')['user_id'].count()

ab_test
interface_eu_test          11567
recommender_system_test     6701
Name: user_id, dtype: int64

In [80]:
#Вычислим сколько пользователей пересекаются и сохраним их в отдельный список

df_pivot = pd.pivot_table(participants, 
                       values='ab_test', 
                       index='user_id', 
                       aggfunc='nunique')
list_users = df_pivot.query('ab_test > 1').reset_index()

print(f'{list_users.user_id.count()} пользователей участвовали в двух группах теста одновременно.')
list_of_users = list_users['user_id'].values.tolist()
print(f'Это составляет {round(len(list_of_users)/eu_users.user_id.count()*100, 1)}% \
от всех участников целевого теста.')
eu_users_list = eu_users.user_id.values.tolist()



1602 пользователей участвовали в двух группах теста одновременно.
Это составляет 25.2% от всех участников целевого теста.


Изучим равномерность их распределения по тестовым группам:

In [71]:
round(
    participants
    .query('ab_test == "recommender_system_test" & user_id.isin(@list_of_users)')['group']
    .value_counts()/
    participants
    .query('ab_test == "recommender_system_test" & user_id.isin(@eu_users_list)')['group']
    .value_counts(),
    2)

A    0.25
B    0.25
Name: group, dtype: float64

Соотношение пользователей теста interface_eu_test, которые попали в тест recommender_system_test, по тестовым группам составляет 25% для каждой из наблюдаемых групп. Они равномерно распределены по тестовым группам и их можно не удалять. Результаты теста от этого не исказятся.

Проанализируем группы пользователей

In [82]:
users_a = (participants
           .query('ab_test == "interface_eu_test" & user_id.isin(@list_of_users) & group == "A"')
           .user_id
           .nunique())
print(f'Количество пользователей теста recommender_system_test, которые попали в группу А теста interface_eu_test: {users_a}.')

Количество пользователей теста recommender_system_test, которые попали в группу А теста interface_eu_test: 819.


In [83]:
users_b = (participants
           .query('ab_test == "interface_eu_test" & user_id.isin(@list_of_users) & group == "B"')
           .user_id
           .nunique())
print(f'Количество пользователей теста recommender_system_test, которые попали в группу В теста interface_eu_test: {users_b}.')

Количество пользователей теста recommender_system_test, которые попали в группу В теста interface_eu_test: 783.


In [84]:
print('Соотношение пользователей теста recommender_system_test, которые попали в тест interface_eu_test, по тестовым группам:')
round(
    participants
    .query('ab_test == "interface_eu_test" & user_id.isin(@list_of_users)')['group']
    .value_counts()/
    participants
    .query('ab_test == "interface_eu_test" & user_id.isin(@eu_users_list)')['group']
    .value_counts(),
    2)

Соотношение пользователей теста recommender_system_test, которые попали в тест interface_eu_test, по тестовым группам:


A    1.0
B    1.0
Name: group, dtype: float64

Пользователи теста interface_eu_test в группах А и В полностью состоят из пользователей, участвующих в тесте recommender_system_test. Это неминуемо отразится на результатах теста.

Посмотрим соотношение пользователей по тестовым группам и равномерность их формирования:

In [99]:
values = [
    (participants
     .query('ab_test == "recommender_system_test" & user_id.isin(@list_of_users) & group == "A"')['group']
     .count()),
    (participants
     .query('ab_test == "recommender_system_test" & user_id.isin(@list_of_users) & group == "B"')['group']
     .count())
]
labels = ['Группа A', 'Группа B']
fig = go.Figure()
fig.add_trace(go.Pie(values=values, labels=labels, hole=.3))
fig.update_layout(title='Соотношение пользователей по тестовым группам')
fig.update_traces(marker=dict(colors=['blue', 'red', 'green']))
fig.show()

Пользователи разделены по тестовым группам не равномерно:

57,5% в группе А,  
42,5% в группе В.


In [94]:
print(f'Относительное различие количества пользователей : {round((1-42.5/57.5)*100, 1)}%.')

Относительное различие количества пользователей: 26.1%.


Проверим пересечение пользователей в группах А и В.

In [103]:
eu_users = eu_users.merge(
    participants.query('ab_test == "recommender_system_test"')[['user_id', 'group']])

print('Количество пользователей группы A в группе B: ', \
      eu_users.query('group == "A"')['user_id'].isin(eu_users.query('group == "B"')['user_id']).sum())
print('Количество пользователей группы B в группе A: ',\
      eu_users.query('group == "B"')['user_id'].isin(eu_users.query('group == "A"')['user_id']).sum())

Количество пользователей группы A в группе B:  0
Количество пользователей группы B в группе A:  0


В группах пересечений нет.  

Проверим, что события происходили в течение 14 дней с момента регистрации пользователя.

In [107]:
events = eu_users[['user_id', 'group']].merge(events, how='left', on='user_id')
events.nunique()

user_id          6351
group_x             2
group_y             2
event_dt        15231
event_name          4
details             4
first_date_x       15
first_date_y       15
dtype: int64

In [108]:
events = events.merge(new_users[['user_id', 'first_date']], how='left', on='user_id')
events_count = len(events)
events = events.query('(event_dt - first_date).dt.days <= 14')
print('Количество событий уменьшилось на {} события.'.format(events_count - len(events)))
events.nunique()

Количество событий уменьшилось на 2870 события.


user_id          3481
group_x             2
group_y             2
event_dt        15231
event_name          4
details             4
first_date_x       15
first_date_y       15
first_date         15
dtype: int64

In [48]:
#проверим название тестов

display(participants.groupby(by = 'ab_test').count())

,user_id,group
ab_test,,
interface_eu_test,11567,11567
recommender_system_test,6701,6701


Согласно данынным есть участники 2 тестов. Оставим, только участников целевого теста recommender_system_test .

In [23]:
participants_raw = participants[participants['ab_test'] == 'recommender_system_test']
participants_raw.head()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


In [24]:
#Оставим пользователей из целевого теста recommender_system_test
df = participants_raw.merge(new_users, on = ['user_id'], how = 'left')

In [25]:
#проверим аудиторию
new_users = new_users[(new_users['first_date'] >= '2020-12-07') & (new_users['first_date'] <= '2020-12-21')]
eu_test = df.groupby(by='region', as_index = False).agg({'user_id':'nunique'})
eu_new = new_users.groupby(by='region', as_index = False).agg({'user_id':'nunique'})
eu_test = int(eu_test[eu_test['region'] == 'EU']['user_id'])
eu_new = int(eu_new[eu_new['region'] == 'EU']['user_id'])

print('Процент новых пользователей составляет: ', round(eu_test/eu_new*100, 2),'%')

Процент новых пользователей составляет:  15.0 %


Процент новых пользователей соответствует техническому заданию.

### Проверка количества участников

In [26]:
df=df[df['region'] == 'EU']
df

,user_id,group,ab_test,first_date,region,device
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC
1,A7A3664BD6242119,A,recommender_system_test,2020-12-20,EU,iPhone
2,DABC14FDDFADD29E,A,recommender_system_test,2020-12-08,EU,Mac
3,04988C5DF189632E,A,recommender_system_test,2020-12-14,EU,iPhone
4,482F14783456D21B,B,recommender_system_test,2020-12-14,EU,PC
...,...,...,...,...,...,...
6346,7C5C12FA1B5AB710,A,recommender_system_test,2020-12-21,EU,Android
6347,91C3969B8A72B908,B,recommender_system_test,2020-12-09,EU,Android
6348,E26F13A65CEAC6EA,A,recommender_system_test,2020-12-17,EU,Mac
6349,95401934D6D6D4FC,B,recommender_system_test,2020-12-13,EU,iPhone


По данным теста мы имеем  6351 пользователей из региона EU

Требования по ожидаемому количеству пользователей выполнены.

In [29]:
#Првоерим попадание маркетинговых событий
marketing_events = pm_events[(pm_events['finish_dt'] >= "2020-12-07") & 
                                          (pm_events['start_dt'] <= "2021-01-04")]
marketing_events

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2020-12-25
10,CIS New Year Gift Lottery,CIS,2020-12-30,2020-12-30


События Christmas&New Year Promo, CIS New Year Gift Lottery попадают в исследуемый период.

# Удостоверимся, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверим динамику набора пользователей в группы теста и проверьте равномерность распределения пользователей по группам теста и корректность их формирования

In [30]:
#выделим данные по второму тесту
int_participants = participants[participants['ab_test'] == 'interface_eu_test']

df2 = df.merge(int_participants, on = ['user_id'])

#удаляем пользователей из контрольной группы, которые входят в группу новой платежной воронки у второго теста
df2 = (df2[~((df2['group_x']=='A')&
            (df2['group_y']=='B'))])

df3=df2[['user_id','group_x','ab_test_y','first_date','region','device']]
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1163 entries, 0 to 1601
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     1163 non-null   object        
 1   group_x     1163 non-null   object        
 2   ab_test_y   1163 non-null   object        
 3   first_date  1163 non-null   datetime64[ns]
 4   region      1163 non-null   object        
 5   device      1163 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 63.6+ KB


In [31]:
#проверим нет ли пользователей, которые участвуют в двух тестах одновременно
df3['user_id'].duplicated().sum()


0

В данных повторяющихся значений по пользователям нет. Переходим к проверке распределения пользователей по группам.

In [32]:
df3.groupby(by='group_x', as_index = False)\
        .agg({'user_id':'count'})

,group_x,user_id
0,A,482
1,B,681


В группе B (новые пользователи) людей немного больше

In [33]:
# добавим события по пользователям в наше общую таблицу
df3 = events.merge(df3, on = ['user_id'], how = 'inner')

df3.columns = ['user_id', 'event_dt', 'event_name', 'details','group', 'ab_test', 'first_date', 'region', 'device']

df3

,user_id,event_dt,event_name,details,group,ab_test,first_date,region,device
0,2B06EB547B7AAD08,2020-12-07 21:36:38,purchase,4.99,A,interface_eu_test,2020-12-07,EU,PC
1,2B06EB547B7AAD08,2020-12-12 07:51:36,purchase,99.99,A,interface_eu_test,2020-12-07,EU,PC
2,2B06EB547B7AAD08,2020-12-22 17:27:15,purchase,4.99,A,interface_eu_test,2020-12-07,EU,PC
3,2B06EB547B7AAD08,2020-12-07 21:36:38,product_cart,NaN,A,interface_eu_test,2020-12-07,EU,PC
4,2B06EB547B7AAD08,2020-12-12 07:51:38,product_cart,NaN,A,interface_eu_test,2020-12-07,EU,PC
...,...,...,...,...,...,...,...,...,...
3734,928364C4C9F13FA8,2020-12-22 20:39:21,login,NaN,B,interface_eu_test,2020-12-21,EU,Android
3735,928364C4C9F13FA8,2020-12-25 01:08:53,login,NaN,B,interface_eu_test,2020-12-21,EU,Android
3736,BEF16764A13AEC34,2020-12-21 03:49:49,login,NaN,B,interface_eu_test,2020-12-21,EU,PC
3737,BEF16764A13AEC34,2020-12-22 18:52:25,login,NaN,B,interface_eu_test,2020-12-21,EU,PC


## Изучение данных о пользовательской активности:

Посмотрим на даты совершения событий участниками теста: 

In [34]:
df3['event_dt'] = df3['event_dt'].dt.date
df3['first_date'] = df3['first_date'].dt.date

In [35]:
print(df3['event_dt'].min())
print(df3['event_dt'].max())

2020-12-07
2020-12-30


События укладываются в заданный период, однако последне событие произошло гораздо раньше окончания периода тестирования

In [36]:
print(df3['first_date'].min())
print(df3['first_date'].max())

2020-12-07
2020-12-21


Даты регистрации укладываются в заданный период

Учтем, что ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 5 процентных пунктов

In [39]:
#Исключим события которые были совершены более 14 дней

df3['event_dt'] = pd.to_datetime(df3['event_dt'])
df3['test'] = df3['first_date']+timedelta(days=14)

df3 = df3[df3['event_dt'] < df3['test']].drop(columns = {'test'})
df3.groupby(by='event_dt')['user_id'].count()

event_dt
2020-12-07    130
2020-12-08    113
2020-12-09    137
2020-12-10    115
2020-12-11     91
2020-12-12     95
2020-12-13    101
2020-12-14    180
2020-12-15    193
2020-12-16    216
2020-12-17    258
2020-12-18    236
2020-12-19    234
2020-12-20    270
2020-12-21    335
2020-12-22    198
2020-12-23    150
2020-12-24    139
2020-12-25     93
2020-12-26    102
2020-12-27     69
2020-12-28    100
2020-12-29     38
Name: user_id, dtype: int64

Данные,выходящие за рамки исследования исключены.

<a id='stage_3_1'></a>
### Промежуточный вывод

Были изучены данные и приведены к необходимым форматам  

Было проверено соответствие требованиям технического задания

Проверне период набора пользователей в тест и удалены лишние данные.  
Ожидаемое количество превышает 6000 участников.  
Соблюдена региональность   
   
Данные подготовлены для дальнейшего анализа.   

<a name="stage_3"></a>
# 3. Исследовательский данных

## Распределение количества событий на пользователя в разрезе групп теста: постройте гистограмму распределения этой величины в разрезе групп и сравните её средние значения между собой у групп теста



## Убедитесь, что время проведения теста не совпадает с маркетинговыми и другими активностями. Настройте автоматическую проверку, выдающую список событий, пересекающихся с тестом. При необходимости оцените воздействие маркетинговых событий на динамику количества событий.  



## Продуктовая воронка: постройте простые продуктовые воронки для двух групп теста с учетом логической последовательности совершения событий; изучите изменение конверсии в продуктовой воронке тестовой группы, по сравнению с контрольной: наблюдается ли ожидаемый эффект увеличения конверсии в группе В, относительно конверсии в группе А?  



## Сделайте общий вывод об изменении пользовательской активности в тестовой группе, по сравнению с контрольной.  

<a name="stage_4"></a>

# 4. Статистический анализ данных

## Средние возраста клиентов, которые пользуются одним и двумя продуктами

Проверим гипотезу о равенстве возрастов клиентов, которые пользуются одним и двумя продуктами:

Нулевая гипотеза - разницы в среднем значении возраста нет.   
Альтернативная - разница в среднем возрасте есть.  

In [38]:
one = df.query('products == 1')['age']
two = df.query('products == 2')['age']

print(f'Средний возраст клиентов, которые пользуются одним продуктом, составляет {one.mean():.2f}')
print(f'Средний возраст клиентов, которые пользуются двумя продуктами - {two.mean():.2f}\n')

alpha = 0.05
print(f'Уровень значимости {alpha:.2%}\n')

pvalue = st.ttest_ind(one, two).pvalue

if pvalue > alpha:
    print(f'P-value = {pvalue:.5f} > {alpha:.4f}')
    print('Не удалось отвергнуть нулевую гипотезу. Разница в средних не статистически значима.')
else:
    print(f'P-value = {pvalue:.5f} <= {alpha:.4f}')
    print('Отвергаем нулевую гипотезу в пользу альтернативной. Разница в средних статистически значима.')

UndefinedVariableError: name 'products' is not defined

Отвергаем нулевую гипотезу, вместо нее принимаем альтернативную. Согласно ей, существует статистически значимая разница в среднем возрасте клиентов, которые пользуются одним и двумя продуктами.

## Доля оттока среди клиентов при наличии и без кредитной карты

Проверим гипотезу о равенстве долей ушедших клиентов, которые брали кредитную карту и которые не брали. Если доли не равны, можно сделать вывод о том, что предложение кредитной карты может уменьшить шанс отткока клиента.

Нулевая гипотеза - доли ушедших равны для бравших и не бравших кредит клиентов.  
Альтернативная - доли не равны.

In [ ]:
count_by_credit = df.groupby('credit_card')['churn'].sum().values # количество успехов
nobs_churn_by_credit = df.groupby('credit_card')['churn'].count().values # количество наблюдений

In [ ]:
wo_churn = count_by_credit[1] / nobs_churn_by_credit[1]
w_churn = count_by_credit[0] / nobs_churn_by_credit[0]

print(f'Доля ухода клиентов без кредитной карты равна {wo_churn:.2%}')
print(f'Доля ухода клиентов с кредитой картой - {w_churn:.2%}\n')


print(f'Уровень значимости {alpha:.2%}')

pvalue = proportions_ztest(count=count_by_credit, nobs=nobs_churn_by_credit)[1]

sign = '>' if pvalue > alpha else '<='
print(f'P-value = {pvalue:.5f} {sign} {alpha:.4f}')
if pvalue > alpha:
    print('Не удалось отвергнуть нулевую гипотезу. Разница в долях не статистически значима.')
else:
    print('Отвергаем нулевую гипотезу в пользу альтернативной. Разница в долях статистически значима.')

Отвергаем нулевую гипотезу. Различие в долях означает, что на данный момент наличие кредита останавливает клиента от ухода.

## Доля наличия кредитной карты для активных и неактивных клиентов

Проверим гипотезу о равенстве долей активных и неактивных клиентов с кредитной картой. Если доля акливных клиентов больше среди обладателей кредитной карты, то предложение оформить ее сможет уменьшить шанс становления пользователя неактивным.  

Нулевая гипотеза - доли клиентов с кредитной картой равны для активных и неактивных пользователей.   
Альтернативная - доли различны.

In [ ]:
last_activity_count_by_credit = df.groupby('last_activity')['churn'].sum().values # количество успехов
last_activity_nobs_by_credit = df.groupby('last_activity')['churn'].count().values # количество наблюдений

In [ ]:
wo_churn = last_activity_count_by_credit[1] / last_activity_nobs_by_credit[1]
w_churn = last_activity_count_by_credit[0] / last_activity_nobs_by_credit[0]

print(f'Доля активных клиентов без кредитной карты равна {wo_churn:.2%}')
print(f'Доля активных клиентов с кредитной картой равна - {w_churn:.2%}\n')


print(f'Уровень значимости {alpha:.2%}')

pvalue = proportions_ztest(count=last_activity_count_by_credit, nobs=last_activity_nobs_by_credit)[1]

sign = '>' if pvalue > alpha else '<='
print(f'P-value = {pvalue:.5f} {sign} {alpha:.4f}')
if pvalue > alpha:
    print('Не удалось отвергнуть нулевую гипотезу. Разница в долях не статистически значима.')
else:
    print('Отвергаем нулевую гипотезу в пользу альтернативной. Разница в долях статистически значима.')

<a id='stage_4_1'></a>
### Промежуточный вывод

Доли клиентов с кредитной картой для активных и неактивных пользователей различны  
Доли ушедших клиентов для клиентов бравших и не бравших крединых карт неравны  
Есть разница в среднем значении возраста, которые пользуются одним и двумя продуктами

Описаны три самых проблемных портрета клиента в диапазоне – от 500 до 2000 юзеров 

<a name="stage_5"></a>

# 5. Выводы и рекомендации

Общий вывод по проведенному исследованию:

- Наибольшее влияние на признак оттока churn оказывают следующие данные:
    - gender — пол. Отточные клиенты чаще мужчины, реже женщины.
    - products - количество продуктов у клиента.Отточные клиенты пользуются большим кол-вом продуктов банка;
    - equity - оценка собственности клиента.У отточных клиентов оценка собственности выше, чем у неотточных;
    - last_activity - активность клиента;
    - age - возраст;
- Отток клиентов по кол-ву продуктов распределен так:
    - чаще всего покидают банк клиенты с 4 продуктами - 63,29%
    - на втором месте клиенты с 5 продуктами - 42,11%
    - реже всего уходят клиенты с 1 продуктом - 7,11%
    - клиенты с 2-3 продуктами находятся в диапазоне 19-29%
    - повышение оттока относительно среднего уровня касается клиентов с 3-5 продуктами
- С увеличением оценки собственности, увеличивается процент оттока клиентов.
    - Минимальное значение при нулевой оценке - 3,52%.
    - Максимальное значение при оценке 9 - 53,85%.
    - Повышение отточности относительно среднего уровня касается клиентов с оценкой собственности 3-9.
    - Наиболее отточными являются клиенты со скорингом 820-920 баллов.

- Меньше всего отток клиентов в группе 65+ лет (3,5 %). Активнее всего теряются кленты сегмента 45-65 лет (20,1% ). Далее в сегменте 25-44 года (18,8 %) и 18-24 года (13,2 %)

- Наиболее отточными клиентами в группах с балансом от 2 до 3 млн. (52,42 %) и от 3 до 4 млн. (59,55 %)

- Наиболее отточными клиентами являются клиенты с заработной платой от 100 тыс. до 200 тыс.(20,47%).

- По результатам статистического анализа принимаем альтернативные гипотезы:   
    1) Существует статистически значимая разница в среднем возрасте клиентов, которые пользуются одним и двумя продуктами.  
    2) Различие в долях означает, что на данный момент наличие кредита останавливает клиента от ухода.  
    3) Рекомендация кредитных карт может увеличить количество активных пользователей

- Сегменты пользователей:

    - 1 портрет. процент оттока 49%
    - 2 портрет. процент оттока 42%
    - 3 портрет. процент оттока 34%